<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [13]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# **Talk2Data**

---

This notebook walks through the initial embeddings setup needed for running the Talk2Data solution. 
As the solution is build on modular components, you can skip to the sections that are relevant for your use case. 

Currently supported Source DBs are: 
- PostgreSQL on Google Cloud SQL 
- BigQuery 

Furthermore, the following vector stores are supported 
- pgvector on PostgreSQL 
- BigQuery vector


The notebook covers the following steps: 
> 1. Setting up PostgreSQL instance on Google Cloud SQL (for using both or either of PostgreSQL & pgvector)

> 2. Migrating BigQuery public data to the PostgreSQL instance (for using PostgreSQL as the Source DB)

> 3. Setting up BigQuery environment (for using both or either of BigQuery Source DB & Vector Store) 

> 4. Populating the vector store with the 'known good' question-SQL pairs (BigQuery vector DB & pgvector)



### 📒 Using this interactive notebook

Click the **run** icons ▶️  of each section within this notebook.

> 💡 Alternatively, you can run the currently selected cell with `Ctrl + Enter` (or `⌘ + Enter` on a Mac).

> ⚠️ **To avoid any errors**, wait for each section to finish in their order before clicking the next “run” icon.

This sample must be connected to a **Google Cloud project**, but nothing else is needed other than your Google Cloud project.

You can use an existing project. Alternatively, you can create a new Cloud project [with cloud credits for free.](https://cloud.google.com/free/docs/gcp-free-tier)

## 🚧 **0. Getting Started**


### 🔐 Authenticate to Google Cloud 
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [14]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Google CLI Auth"""
# !gcloud auth application-default login


import google.auth
credentials, project_id = google.auth.default()

### 🔗 Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook. 

In [15]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
# PROJECT_ID = "talk2data-genai-sa" #@param {type:"string"}
PROJECT_ID = input("Please enter the Google Cloud Project ID ")

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
PROJECT_ID
# !gcloud auth application-default set-quota-project {PROJECT_ID}

Updated property [core/project].


'talk2data-genai-sa'

In [16]:
import os
os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

### ⚙️ Set up Configuration 
The notebook will load all the configurations from the `config.ini` file in the root directory. 
If you ran the initial notebook `1_setup_envs.ipynb`, most of the parameters are already stored in the config file. 
Use the below cells to retrieve these values and specify additional ones required for this notebook. 

In [17]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

import configparser
config = configparser.ConfigParser()
config.read(module_path+'/config.ini')

PROJECT_ID = config['GCP']['PROJECT_ID']
DATA_SOURCE = config['CONFIG']['DATA_SOURCE']
VECTOR_STORE = config['CONFIG']['VECTOR_STORE']
PG_SCHEMA = config['PGCLOUDSQL']['PG_SCHEMA']
PG_DATABASE = config['PGCLOUDSQL']['PG_DATABASE']
PG_USER = config['PGCLOUDSQL']['PG_USER']
PG_REGION = config['PGCLOUDSQL']['PG_REGION'] 
PG_INSTANCE = config['PGCLOUDSQL']['PG_INSTANCE'] 
PG_PASSWORD = config['PGCLOUDSQL']['PG_PASSWORD']
BQ_TALK2DATA_DATASET_NAME = config['BIGQUERY']['BQ_TALK2DATA_DATASET_NAME']
BQ_LOG_TABLE_NAME = config['BIGQUERY']['BQ_LOG_TABLE_NAME'] 
BQ_DATASET_REGION = config['BIGQUERY']['BQ_DATASET_REGION']
BQ_DATASET_NAME = config['BIGQUERY']['BQ_DATASET_NAME']
BQ_TABLE_LIST = config['BIGQUERY']['BQ_TABLE_LIST']

In [18]:
#@markdown Please fill in the values below and then run the cell.
# EXAMPLES = False
EXAMPLES = input("Do you have examples in the csv in the nedded folder answer in True or False").lower()
if EXAMPLES=="True":
    EXAMPLES=True
else:
    EXAMPLES=False
# Quick input validations.
# assert EXAMPLES, "⚠️ Please specify whether to include the Known Good SQL pairs."


## 📑 **1. Create embeddings**
The following code will fetch the table and column schema along with the known good SQL examples (the working question-SQL pairs we uploaded in the previous notebooks from the file `known_good_sql.csv`) from the specified source Database. 

Supported Source DBs are: 
- `cloudsql-pg`
- `bigquery`

In [19]:
#Based on the source we need to provide Schema variable to the retrieve metadata embedding. For bigquery its dataset name and for Postgres its schema name
from dbconnectors import pgconnector, bqconnector

if DATA_SOURCE=='bigquery':
    USER_DATABASE=BQ_DATASET_NAME 
    src_connector = bqconnector
else: 
    USER_DATABASE=PG_SCHEMA
    src_connector = pgconnector

print("Source selected is : "+ str(DATA_SOURCE) + "\nSchema or Dataset Name is : "+ str(USER_DATABASE))

Source selected is : bigquery
Schema or Dataset Name is : fda_food


### Retrieve Embeddings

In [22]:
from embeddings.retrieve_embeddings import retrieve_embeddings 

table_schema_embeddings, col_schema_embeddings, example_embeddings = retrieve_embeddings(DATA_SOURCE, EXAMPLES,SCHEMA=USER_DATABASE)


LLM generated 1 Table Descriptions

LLM generated 7 Column Descriptions


In [ ]:
table_schema_embeddings

In [ ]:
col_schema_embeddings

In [ ]:
example_embeddings

## 🗃️ **2. Store Embeddings**
The following code will store the retrieved embedding DataFrames in the specified Vector Store.

Supported Vector Stores are: 
- `cloudsql-pgvector`
- `bigquery-vector`

In [23]:
print("Vector Store selected is : "+ str(VECTOR_STORE))

if VECTOR_STORE=='bigquery-vector':
    instance_name=None
    database_name=BQ_TALK2DATA_DATASET_NAME
    schema=BQ_DATASET_NAME
    database_user=None
    database_password=None
    region=BQ_DATASET_REGION
    vector_connector = bqconnector
    call_await = False

else:
    instance_name=PG_INSTANCE
    database_name=PG_DATABASE
    schema=PG_SCHEMA
    database_user=PG_USER
    database_password=PG_PASSWORD
    region=PG_REGION
    vector_connector = pgconnector
    call_await=True



Vector Store selected is : cloudsql-pgvector


In [24]:
from embeddings.store_embeddings import store_schema_embeddings

await(store_schema_embeddings(table_details_embeddings=table_schema_embeddings, 
                            tablecolumn_details_embeddings=col_schema_embeddings, 
                            project_id=PROJECT_ID,
                            instance_name=instance_name,
                            database_name=database_name,
                            schema=schema,
                            database_user=database_user,
                            database_password=database_password,
                            region=region,
                            EXAMPLES = EXAMPLES, 
                            VECTOR_STORE = VECTOR_STORE,
                            example_prompt_sql_embeddings=example_embeddings 
                            ))

'Embeddings are stored successfully'

## ▶️ **3. Run the Talk2Data Pipeline**

In [25]:
print("\033[1mData Source :- "+ DATA_SOURCE)
print("Vector Store :- "+ VECTOR_STORE)
print("Schema :- "+USER_DATABASE)


num_similar_matches = 10 
similarity_score_matches = 0.3

RUN_DEBUGGER = True 
EXECUTE_FINAL_SQL = True 

if DATA_SOURCE== "cloudsql-pg":
    user_question = "How many asian men were part of the leadership workforce in 2021?"
elif DATA_SOURCE == "bigquery":
    user_question = "What are the top 5 cities with highest recalls?"

prompt_for_question = "Please enter your question for source :" + DATA_SOURCE + " and database : " + USER_DATABASE
# user_question = input(prompt_for_question) #Uncomment if you want to ask question yourself


print("Asked Question :- "+user_question)

Data Source :- bigquery
Vector Store :- cloudsql-pgvector
Schema :- fda_food
Asked Question :- What are the top 5 cities with highest recalls?


In [26]:
from google.api_core.exceptions import NotFound
from google.cloud import aiplatform
import pandas as pd
import vertexai

from agents import EmbedderAgent, BuildSQLAgent, DebugSQLAgent, ValidateSQLAgent, ResponseAgent


embedder = EmbedderAgent('vertex') 

SQLBuilder = BuildSQLAgent('gemini-pro')
SQLChecker = ValidateSQLAgent('gemini-pro')
SQLDebugger = DebugSQLAgent()
Responder = ResponseAgent('gemini-pro')

AUDIT_TEXT=''
found_in_vector = 'N'
process_step=''
final_sql='Not Generated Yet'

vertexai.init(project=PROJECT_ID, location=region)
aiplatform.init(project=PROJECT_ID, location=region)


In [27]:
# Fetch the embedding of the user's input question 
embedded_question = embedder.create(user_question)
AUDIT_TEXT = AUDIT_TEXT + "User Question : " + str(user_question) + "\nUser Database : " + str(USER_DATABASE)
process_step = "Get Exact Match"
# Look for exact matches in known questions 
exact_sql_history = vector_connector.getExactMatches(user_question) 

if exact_sql_history is not None:
    found_in_vector = 'Y' 
    final_sql = exact_sql_history
    invalid_response = False
    AUDIT_TEXT = AUDIT_TEXT + "\nExact match has been found" 


else:
    # No exact match found. Proceed looking for similar entries in db 
    AUDIT_TEXT = AUDIT_TEXT + "\nNo exact match found looking for similar entries and schema"
    process_step = "Get Similar Match"
    if call_await:
        similar_sql = await vector_connector.getSimilarMatches('example', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)
    else:
        similar_sql = vector_connector.getSimilarMatches('example', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)

    process_step = "Get Table and Column Schema"
    # Retrieve matching tables and columns
    if call_await: 
        tables_schema =  await vector_connector.getSimilarMatches('table', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)
        tables_detailed_schema =  await vector_connector.getSimilarMatches('column', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)
    else:
        tables_schema =  vector_connector.getSimilarMatches('table', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)
        tables_detailed_schema =  vector_connector.getSimilarMatches('column', USER_DATABASE, embedded_question, num_similar_matches, similarity_score_matches)

    AUDIT_TEXT = AUDIT_TEXT + "\n Retrived Similar Entries, Table Schema and Column Schema"
    # If similar table and column schemas found: 
    if len(tables_schema.replace('Schema(values):','').replace(' ','')) > 0 or len(tables_detailed_schema.replace('Column name(type):','').replace(' ','')) > 0 :

        # GENERATE SQL
        process_step = "Build SQL"
        generated_sql = SQLBuilder.build_sql(DATA_SOURCE,user_question,tables_schema,tables_detailed_schema,similar_sql)
        final_sql=generated_sql
        AUDIT_TEXT = AUDIT_TEXT + "\n Generated SQL : " + str(generated_sql)
        
        if 'unrelated_answer' in generated_sql :
            invalid_response=True

        # If agent assessment is valid, proceed with checks  
        else:
            invalid_response=False

            if RUN_DEBUGGER: 
                generated_sql, invalid_response,AUDIT_TEXT = SQLDebugger.start_debugger(DATA_SOURCE,generated_sql, user_question, SQLChecker, tables_schema, tables_detailed_schema, similar_sql) 

            final_sql=generated_sql
            AUDIT_TEXT = AUDIT_TEXT + "\nFinal SQL after Debugger : " +str(final_sql)


    # No matching table found 
    else:
        invalid_response=True
        print('No tables found in Vector ...')
        AUDIT_TEXT = AUDIT_TEXT + "\n No tables have been found in the Vector DB..."
        

No exact match found for the user prompt
Did not find any results. Adjust the query parameters.

 SQL Syntax Validity:True

 SQL Syntax Error Description:[]

This query will process 612376 bytes.
exec_result_df:This query will process 612376 bytes.


In [28]:
if not invalid_response:
    try: 
        if EXECUTE_FINAL_SQL is True:
                final_exec_result_df=src_connector.retrieve_df(final_sql.replace("```sql","").replace("```","").replace("EXPLAIN ANALYZE ",""))
                print('\n Question: ' + user_question + '\n')
                # print('\n Final SQL Execution Result: \n')
                # print(final_exec_result_df)
                response = final_exec_result_df
                _resp=Responder.run(user_question, response)
                AUDIT_TEXT = AUDIT_TEXT + "\n Model says " + str(_resp) 


        else:  # Do not execute final SQL
                print("Not executing final SQL since EXECUTE_FINAL_SQL variable is False\n ")
                response = "Please enable the Execution of the final SQL so I can provide an answer"
                _resp=Responder.run(user_question, response)
                AUDIT_TEXT = AUDIT_TEXT + "\n Model says " + str(_resp) 

    except ValueError: 
          print('')
    # except Exception as e: 
    #     print(f"An error occured. Aborting... Error Message: {e}")
        
else:  # Do not execute final SQL
    print("Not executing final SQL as it is invalid, please debug!")
    response = "I am sorry, I could not come up with a valid SQL."
    _resp=Responder.run(user_question, response)
    AUDIT_TEXT = AUDIT_TEXT + "\n Model says " + str(_resp)

print("Final Answer :" + str(_resp))
bqconnector.make_audit_entry(DATA_SOURCE, USER_DATABASE, "gemini-pro", user_question, final_sql, found_in_vector, "", process_step, "", AUDIT_TEXT)  


 Question: What are the top 5 cities with highest recalls?


 Final SQL Execution Result: 

Final Answer :The top 5 cities with the highest number of recalls are:

1. Chicago with 870 recalls
2. Milwaukee with 637 recalls
3. Baltimore with 443 recalls
4. Austin with 382 recalls
5. Troy with 370 recalls


'Log Row added'

## 🗑 Clean Up Notebook Resources
Make sure to delete your Cloud SQL instance and BigQuery Dataset when your are finished with this notebook to avoid further costs. 💸 💰

In [ ]:
# delete Cloud SQL instance
!gcloud sql instances delete {PG_INSTANCE} -q

#delete BigQuery Dataset using bq utility
!bq rm -r -f -d {BQ_DATASET_NAME}

#delete BigQuery Talk2Data Dataset Processing Dataset using bq utility

!bq rm -r -f -d {BQ_TALK2DATA_DATASET_NAME}

